# Convolutionl Neural Network

In this notebook we try to use CNN for instrument classification

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
data = pd.read_csv('data.csv')

# Remove index
data = data.drop('Unnamed: 0', axis=1)

# 1-hotify instrument
instrdummy = pd.get_dummies(data.instrument)
instrdummy

for c in instrdummy.columns:
    data[c] = instrdummy[c]
    
data = data.drop('instrument', axis=1)

Set the number of samples in the data

In [ ]:
smpls = 30

Here is our neural network

In [ ]:
SCrest_input = keras.Input(shape=(smpls,), name='SCrest')
SCentroid_input = keras.Input(shape=(smpls,), name='SCentroid')
SKurtosis_input = keras.Input(shape=(smpls,), name='SKurtosis')
SMean_input = keras.Input(shape=(smpls,), name='SMean')
SRolloff_input = keras.Input(shape=(smpls,), name='SRolloff')
SVariance_input = keras.Input(shape=(smpls,), name='SVariance')
SSkewness_input = keras.Input(shape=(smpls,), name='SSkewness')
SFlux_input = keras.Input(shape=(smpls,), name='SFlux')
mfcd_input = keras.Input(shape=(smpls,), name='CeptralFeatures')
zcr_input = keras.Input(shape=(smpls,), name='TemporalFeatures')
inps = [SCrest_input, SCentroid_input, SKurtosis_input, SMean_input, SRolloff_input, SVariance_input, SSkewness_input, SFlux_input, mfcd_input, zcr_input]

def soloLayer(inputlayer):
    r = layers.Reshape((smpls,1))(inputlayer)
    conv1 = layers.Conv1D(filters=128, kernel_size=5)(r)
    conv2 = layers.Conv1D(filters=64, kernel_size=3)(conv1)
    max11 = layers.MaxPooling1D(3)(conv2)
    batch2 = layers.BatchNormalization()(max11)
    
    return batch2

def merge(ls):
    flats = [layers.Flatten()(l) for l in ls]
    
    return layers.concatenate(flats)

input_merged = merge([soloLayer(il) for il in inps])

d2 = layers.Dense(64, 'tanh')(input_merged)
d3 = layers.Dense(32)(d2)
dr2 = layers.Dropout(.05)(d3)

out = layers.Dense(8, 'softmax')(dr2)

model = keras.Model(
    inputs=inps+[timbral_input],
    outputs=out
)

model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["acc"]
)

Split the data into separate dataframes

In [ ]:
from sklearn.model_selection import train_test_split
train_scre, test_scre, train_scen, test_scen, train_skur, test_skur, train_smea, test_smea, train_srol, test_srol, train_svar, test_svar, train_sskew, test_sskew, train_sflu, test_sflu, train_mfcd, test_mfcd, train_zcr, test_zcr, train_y, test_y = train_test_split(
    data.SCrest,
    data.SCentroid,
    data.SKurtosis,
    data.SMean,
    data.SRolloff,
    data.SVariance,
    data.SSkewness,
    data.SFlux,
    data.MFCD,
    data.ZCR,
    data[['cel', 'cla', 'flu', 'gac', 'pia', 'sax', 'tru', 'vio']],
    test_size=0.2
)

train_X = [train_scre, train_scen, train_skur, train_smea, train_srol, train_svar, train_sskew, train_sflu, train_mfcd, train_zcr]
test_X = [test_scre, test_scen, test_skur, test_smea, test_srol, test_svar, test_sskew, test_sflu, test_mfcd, test_zcr]

Make the data suitable for Keras consumption

In [ ]:
from keras import backend as K

train_X_list = []
for x in train_X:
    train_X_list += [np.asarray([K.cast_to_floatx(a) for a in x.values])]
    
train_y_list = K.cast_to_floatx(train_y.values.astype('float32'))

test_X_list = []
for x in test_X:
    test_X_list += [np.asarray([K.cast_to_floatx(a) for a in x.values])]
    
test_y_list = K.cast_to_floatx(test_y.values.astype('float32'))

Make so that the Tensorflow functions run eagerly

In [ ]:
tf.config.run_functions_eagerly(True)

Teach the model

In [ ]:
model.fit(
    x=train_X_list, 
    y=train_y_list, 
    batch_size=128, epochs=17,
    validation_data=(test_X_list, test_y_list)
)

Create a confusion matrix

In [ ]:
y_pred = np.argmax(model.predict(test_X_list), axis=1)
y_actual = np.argmax(test_y_list, axis=1)
cm = confusion_matrix(y_actual, y_pred)

In [ ]:
df_cm = pd.DataFrame(cm, index = [i for i in ['cel', 'cla', 'flu', 'gac', 'pia', 'sax', 'tru', 'vio']],
                       columns = [i for i in ['cel', 'cla', 'flu', 'gac', 'pia', 'sax', 'tru', 'vio']])

plt.figure(figsize = (11,9))
sn.heatmap(df_cm, annot=True)